<a href="https://colab.research.google.com/github/Saaj369/Diffusion-Limited-Aggregation/blob/main/Langevin_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diffusion LImited aggregation with killing particles method

In [ ]:
# Imports
import numpy as np
from math import pi, sqrt
%matplotlib widget

In [ ]:
# Data structure for storing aggregate data
class cluster():
    def __init__(self,master,children,id):
        self.master = master
        self.children = children
        self.id = id
    def add(self,children):
        self.children.append(children)
    def reveal(self):
        print(self.id)
        print(self.children)


In [ ]:
# parameters
m = 0.2; R = 0.4; eta = 0.01; kT = 2; gmma = 6*pi*eta*R/m; dt = 0.1 #n = 90000 # Depreciated n
mu = 0; rc = 0.1; N = 5000 # NOP aggregating
L = 10

In [ ]:
# main class for performing DLA
# With Periodic Boundary Condition and seed Aggregation for 1 particle
class BAOAB():
    def __init__(self,initials,seed:cluster):
        self.seed = seed
        self.aggs = [seed]
        self.sr = 2*R
        self.meanr = None
        # self.L = 10*self.sr
        # self.ts = np.zeros(n)
        # self.xs = np.zeros((n, 3))
        # self.vs = np.zeros((n, 3))
        self.ts, self.xs, self.vs = [0],[0],[0]
        self.ts[0], self.xs[0], self.vs[0] = initials[0], initials[1], initials[2]

    def F(self,t):
        ''' This is any external deterministic force'''
        return 0

    def B(self,vt,ft):
        nvt = vt + ft*dt/(2*m)
        return nvt

    def A(self,xt,vt):
        nxt = xt + vt*(dt/2)
        return nxt

    def O(self,vt):
        Rt = np.random.normal(0, 1, size=3)
        nvt = np.exp(-gmma*dt)*vt + Rt*sqrt(kT/m)*sqrt(1-np.exp(-2*gmma*dt))
        return nvt

    def check_for_aggregation(self,xt):
        seed = self.seed
        master = seed.master
        children = seed.children
        for member in self.aggs:
            r = xt - member.master
            if np.linalg.norm(r) <= 2*R:
                ru = r/np.linalg.norm(r)
                rn = member.master + ru
                ob = cluster(master = rn, children = [], id = None)
                member.children.append(ob)
                self.aggs.append(ob)
                sr = np.linalg.norm(rn - master) + 2*R
                if sr > self.sr:
                    self.sr = sr # changing Seed radius
                return True
        return False


    def check_for_closeness(self,xt):
        r = np.linalg.norm(xt - self.seed.master)
        if r <= self.sr:
            return -1
        # elif r >= 20*self.sr:
        elif r >= ((20*self.meanr)+self.sr):
            return 1
        else:
            return 0


    def execute(self):
        self.xs[0] = self.gen_particle()
        # L = 10*self.sr
        i = 1
        count = 0
        while count < N:
            vt, xt = self.vs[i-1], self.xs[i-1]
            t = self.ts[i-1] + i*dt
            ft = self.F(t)

            #B
            vt = self.B(vt,ft)
            #A
            xt = self.A(xt,vt)
            #O
            vt = self.O(vt)
            #A
            xt = self.A(xt,vt)
            #B
            vt = self.B(vt,ft)


            # self.ts[i], self.xs[i], self.vs[i] = t, xt, vt
            self.ts.append(t); self.xs.append(xt); self.vs.append(vt)
            check = self.check_for_closeness(xt)
            if check == -1:
                if self.check_for_aggregation(xt):
                    print(f"{count} Aggregated at {i}")
                    # self.save_data(f"LD2//P{count}.xyz")
                    self.initiate()
                    i = 0
                    count += 1
            if check == 1:
                # print(f"{count} Particle went outside: KILLING")
                self.initiate()
                i = 0

            i += 1
    def initiate(self):
        x0 = self.gen_particle()
        # print(x0)
        self.ts, self.xs, self.vs = [0],[0],[0]
        self.ts[0], self.xs[0], self.vs[0] = 0, x0, np.array([0,0,0])
        # L = self.sr*10

    def gen_particle(self):
        r = np.array([np.random.uniform(-1, 1),np.random.uniform(-1, 1),np.random.uniform(-1, 1)])
        r = self.seed.master + ((10*self.meanr)+self.sr)* r/np.linalg.norm(r)
        return r

    def estimate_meanr(self):
        x0 =np.array([np.random.uniform(-1, 1),np.random.uniform(-1, 1),np.random.uniform(-1, 1)])
        print(x0)
        self.ts, self.xs, self.vs = [0],[0],[0]
        self.ts[0], self.xs[0], self.vs[0] = 0, x0, np.array([0,0,0])
        mean = 0
        for i in range(1,1000):
            vt, xt = self.vs[i-1], self.xs[i-1]
            t = self.ts[i-1] + i*dt
            ft = self.F(t)

            #B
            vt = self.B(vt,ft)
            #A
            xt = self.A(xt,vt)
            #O
            vt = self.O(vt)
            #A
            xt = self.A(xt,vt)
            #B
            vt = self.B(vt,ft)

            self.ts.append(t); self.xs.append(xt); self.vs.append(vt)
            mean += np.linalg.norm(self.xs[i] - self.xs[i-1])
        mean = mean/999
        self.meanr = mean

    def save_data(self,filename):
        with open(filename, 'w') as f:
            for i in range(len(self.ts)):
                f.write(f"{1}\n")  # Number of particles
                f.write(f"time = {self.ts[i]} step = {i}\n")  # Optional comment line
                _id = 0
                # for position in self.POS:
                particle_id = f'p_{_id}'
                # x, y, z = self[particle_id]
                f.write(f"X{particle_id} {self.xs[i][0]} {self.xs[i][1]} {self.xs[i][2]}\n")

    def save_agg(self,filename):
        POS = np.array([p.master for p in self.aggs])
        with open(filename, 'w') as f:
            f.write(f"{len(POS)}\n")  # Number of particles
            # f.write(f"time = {self.ts[i]} step = {i}\n")  # Optional comment line
            for i in range(len(POS)):
                particle_id = f'p_{i}'
            # x, y, z = self[particle_id]
                f.write(f"X{particle_id} {POS[i][0]} {POS[i][1]} {POS[i][2]}\n")



    def plot(self):
        import numpy as np
        import matplotlib.pyplot as plt

        # Assuming your data is stored in a numpy array named 'data'
        # Each row represents a 3D vector (x, y, z) of particle position
        # Example data with 100 time steps and 3 dimensions

        # Create subplots for each 2D projection
        fig, axs = plt.subplots(1, 3, figsize=(11, 4))
        ts, xs,vs = np.array(self.ts), np.array(self.xs), np.array(self.vs)

        # Plot XY plane (x vs y)
        axs[0].plot(xs[:, 0], xs[:, 1])
        axs[0].set_xlabel('X')
        axs[0].set_ylabel('Y')
        axs[0].set_title('XY Plane')

        # Plot YZ plane (y vs z)
        axs[1].plot(xs[:, 1], xs[:, 2])
        axs[1].set_xlabel('Y')
        axs[1].set_ylabel('Z')
        axs[1].set_title('YZ Plane')

        # Plot XZ plane (x vs z)
        axs[2].plot(xs[:, 0], xs[:, 2])
        axs[2].set_xlabel('X')
        axs[2].set_ylabel('Z')
        axs[2].set_title('XZ Plane')

        # Adjust layout and show plot
        plt.tight_layout()
        plt.show()
        # plt.close()
    def plot3d(self):
        import numpy as np
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D

        # Assuming you have position data in a numpy array 'positions'
        # Each row of 'positions' contains the x, y, and z coordinates of the particle at different time steps

        # Example data (replace this array with your actual data)
        # Let's assume 'positions' is a 2D numpy array with shape (time_steps, 3)

        # Create a 3D plot
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Extract x, y, and z coordinates from the 'positions' array
        xs = np.array(self.xs)
        x = xs[:, 0]
        y = xs[:, 1]
        z = xs[:, 2]

        # Plot the trajectory with smaller dots (adjust the markersize as needed)
        ax.plot(x, y, z, marker='o', markersize=1)

        # Set labels and title
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Particle Trajectory')

        # Show plot
        plt.show()
    def show_aggregate(self,s=100):
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D

        # Enable interactive mode
        %matplotlib notebook

        # Extract x, y, z coordinates from positions array
        POS = np.array([p.master for p in self.aggs])
        x = POS[:, 0]
        y = POS[:, 1]
        z = POS[:, 2]


        # Create a 3D scatter plot
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the particles
        ax.scatter(x, y, z, c='r',s=s, marker='o', label='Particles')

        # Set labels and title
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Particle Positions')

        # Add legend
        ax.legend()

        # Show the plot
        plt.show()

In [ ]:
# Creating data structure object
seed = cluster(master = np.array([0,0,0]), children = [], id = 0)

In [ ]:
# Initilisnig initial conditions and creating object
t0 = 0
x0 = np.array([0,0,0]) # dummy
v0 = np.array([0,0,0])
initial = (t0,x0,v0)
obj = BAOAB(initial,seed)

In [ ]:
# estimating mean free path and initiating
obj.estimate_meanr()
obj.initiate()

In [ ]:
# Executing simulation
obj.execute()

In [ ]:
# Saving file
def save_agg(filename = "agg5000_killing_00_eta.xyz"):
    POS = np.array([p.master for p in obj.aggs])
    with open(filename, 'w') as f:
        f.write(f"{len(POS)}\n")  # Number of particles
        f.write(f"m = 0.2; R = 0.4; eta = 0.01; kT = 2; dt = 0.1 \n")  # Optional comment line
        for i in range(len(POS)):
            particle_id = f'p_{i}'
        # x, y, z = self[particle_id]
            f.write(f"X{particle_id} {POS[i][0]} {POS[i][1]} {POS[i][2]}\n")

save_agg()

# Data analysis from file

In [ ]:
import numpy as np

# Read the file and skip the first two lines
with open('agg5000_killing_00_eta.xyz', 'r') as f:
    lines = f.readlines()[2:]

# Extract position data from each line and store in a NumPy array
apos = np.array([list(map(float, line.split()[1:])) for line in lines])

# Print the extracted positions
print(apos)

In [ ]:
# Defining various functions, names are self understood
from math import pi
def Nr(r):
    norms = np.linalg.norm(apos, axis=1)
    counts = np.sum(norms <= r)
    return counts-1

def span():
    norms = np.linalg.norm(apos, axis=1)
    return np.max(norms)

def Cr(r,dr):
    res = Nr(r+dr) - Nr(r-dr)
    res = res/(1000*8*pi*dr*r**2)
    return res


In [ ]:
# Extracting and preparing data
rs = np.arange(1, 6, 0.1)
crs = []
Nrs = []
for r in rs:
    crs.append(Cr(r,0.5))
    Nrs.append(Nr(r))
crs = np.array(crs)
Nrs = np.array(Nrs)

In [ ]:
# PLotting stuff
import numpy as np
import matplotlib.pyplot as plt

# Take the natural logarithm (ln) of both arrays
ln_rs = np.log(rs[1:])
ln_crs = np.log(crs[1:])
ln_Nrs = np.log(Nrs[1:])

# Plot the ln of crs and perform linear fit
plt.figure(figsize=(8, 4))
plt.plot(ln_rs, ln_crs, marker='o', linestyle='-')
slope_crs, intercept_crs = np.polyfit(ln_rs, ln_crs, 1)
plt.xlabel('ln(r)')
plt.ylabel('ln(C(r))')
plt.title('ln(r) Vs ln(C(r))')
plt.grid(True)

# Plot the linear fit line
plt.plot(ln_rs, slope_crs * ln_rs + intercept_crs, color='red', linestyle='--')

# Print the slope for ln(C(r))
print("Slope of the linear fit for ln(C(r)):", slope_crs)

plt.show()

# Plot the ln of Nrs and perform linear fit
plt.figure(figsize=(8, 4))
plt.plot(ln_rs, ln_Nrs, marker='o', linestyle='-')
slope_Nrs, intercept_Nrs = np.polyfit(ln_rs, ln_Nrs, 1)
plt.xlabel('ln(r)')
plt.ylabel('ln(N(r))')
plt.title('ln(r) Vs ln(N(r))')
plt.grid(True)

# Plot the linear fit line
plt.plot(ln_rs, slope_Nrs * ln_rs + intercept_Nrs, color='red', linestyle='--')

# Print the slope for ln(N(r))
print("Slope of the linear fit for ln(N(r)):", slope_Nrs)

plt.show()


# THANK YOU